In [1]:
from pyspark import SparkContext
from pyspark.sql.functions import *
sc = SparkContext("local[2]", "KafkaTest")
sc.version

'3.0.1'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [3]:


brokers = "kafka1:9092,kafka2:9092"

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()



In [4]:
spark.version

'3.0.1'

In [5]:
from pyspark.sql.types import *
schema = StructType([
    StructField("count", IntegerType()),
    StructField("time", StringType())])



In [6]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers",brokers) \
    .option("subscribe","Tut01") \
    .option("startingOffsets", "earliest") \
    .load()

ds = df.selectExpr("CAST (key AS STRING)", "CAST(value AS STRING)", "timestamp")
ds.writeStream.queryName("temp2").format("memory").start()
spark.sql("select * from  temp2 limit 10").show()

+---+-----+---------+
|key|value|timestamp|
+---+-----+---------+
+---+-----+---------+



In [7]:
inputstream = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers",brokers) \
  .option("subscribe", "Tut01") \
  .option("auto.offset.reset", "latest") \
  .option("startingOffsets", "earliest") \
  .load()


In [8]:
display(inputstream)

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [9]:
import json
# data = dsraw.select(json.loads(col("value").cast("string"), schema))
ds =inputstream.selectExpr("CAST(value AS STRING)")

In [10]:
rawQuery = ds \
        .writeStream \
        .queryName("qraw")\
        .format("memory")\
        .outputMode("append") \
        .start()
        


In [11]:
raw = spark.sql("select * from qraw")
raw.show()

+-----+
|value|
+-----+
+-----+



In [12]:
from time import sleep
for x in range(5):
    spark.table("qraw").show()
    sleep(5)

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+



In [ ]:
from IPython.display import display, clear_output
import time 
while True:
    clear_output(wait=True)
    display(rawQuery.status )
    display(spark.sql("select * from qraw").show())
    time.sleep(5)

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

+--------------------+
|               value|
+--------------------+
|{"count": 0, "tim...|
|{"count": 1, "tim...|
|{"count": 2, "tim...|
|{"count": 3, "tim...|
|{"count": 4, "tim...|
|{"count": 5, "tim...|
|{"count": 6, "tim...|
|{"count": 7, "tim...|
|{"count": 8, "tim...|
|{"count": 9, "tim...|
|{"count": 10, "ti...|
|{"count": 11, "ti...|
|{"count": 12, "ti...|
|{"count": 13, "ti...|
|{"count": 14, "ti...|
|{"count": 15, "ti...|
|{"count": 16, "ti...|
|{"count": 17, "ti...|
|{"count": 18, "ti...|
|{"count": 19, "ti...|
+--------------------+
only showing top 20 rows



None

In [ ]:
print("===============================================================")
query  = inputstream \
      .selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")\
      .writeStream\
      .outputMode("append")\
      .format("console")\
      .start()

print("===============================================================> " +str(query.isActive))
query.awaitTermination()

In [ ]:
alertQuery = ds \
    .writeStream \
        .queryName("qalerts")\
        .format("memory")\
        .start()

In [7]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import statistics


KAFKA_TOPIC = "Stream_Test"


#   spark = SparkSession \
#         .builder \
#         .appName("PySpark Structured Streaming with Kafka Demo") \
#         .master("local[*]") \
#         .config("spark.jars", "/home/cldr/streams-dev/libs/spark-sql-kafka-0-10_2.11-2.4.4.jar,/home/cldr/streams-dev/libs/kafka-clients-2.0.0.jar") \
#         .config("spark.executor.extraClassPath", "/home/cldr/streams-dev/libs/spark-sql-kafka-0-10_2.11-2.4.4.jar:/home/cldr/streams-dev/libs/kafka-clients-2.0.0.jar") \
#         .config("spark.executor.extraLibrary", "/home/cldr/streams-dev/libs/spark-sql-kafka-0-10_2.11-2.4.4.jar:/home/cldr/streams-dev/libs/kafka-clients-2.0.0.jar") \
#         .config("spark.driver.extraClassPath", "/home/cldr/streams-dev/libs/spark-sql-kafka-0-10_2.11-2.4.4.jar:/home/cldr/streams-dev/libs/kafka-clients-2.0.0.jar") \
#         .getOrCreate()

spark.sparkContext.setLogLevel("DEBUG")

schema = StructType() \
        .add("WheelAngle", IntegerType()) \
        .add("acceleration", IntegerType()) \
        .add("heading", IntegerType()) \
        .add("reading_time", IntegerType()) \
        .add("tractionForce", IntegerType()) \
        .add("vel_latitudinal", IntegerType()) \
        .add("vel_longitudinal", IntegerType()) \
        .add("velocity", IntegerType()) \
        .add("x_pos", IntegerType()) \
        .add("y_pos", IntegerType()) \
        .add("yawrate", IntegerType())


 # Construct a streaming DataFrame that reads from testtopic
trans_det_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", brokers) \
        .option("subscribe", KAFKA_TOPIC) \
        .option("startingOffsets", "latest") \
        .load() \
        .selectExpr("CAST(value as STRING)", "CAST(timestamp as STRING)","CAST(topic as STRING)")

print("Status " + str(trans_det_df.isStreaming))

#(from_json(col("value").cast("string"),schema))

    #Q1 =  trans_det_df.select(from_json(col("value"), schema).alias("parsed_value"), "timestamp")
    #Q2 =  trans_det_d.select("parsed_value*", "timestamp")


# query = trans_det_df.writeStream \
#             .format("console") \
#             .option("truncate","false") \
#             .start() \
#             .awaitTermination()

query = trans_det_df \
    .writeStream \
    .format("memory") \
    .queryName("testData2") \
    .outputMode("append") \
    .start()

Status True


In [24]:
from pyspark.sql.types import *
import statistics
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *


brokers = "kafka1:9092,kafka2:9092"

spark = SparkSession \
    .builder \
    .appName("StructuredStreamingKafka") \
    .getOrCreate()

KAFKA_TOPIC = "Stream_Test"


#   spark = SparkSession \
#         .builder \
#         .appName("PySpark Structured Streaming with Kafka Demo") \
#         .master("local[*]") \
#         .config("spark.jars", "/home/cldr/streams-dev/libs/spark-sql-kafka-0-10_2.11-2.4.4.jar,/home/cldr/streams-dev/libs/kafka-clients-2.0.0.jar") \
#         .config("spark.executor.extraClassPath", "/home/cldr/streams-dev/libs/spark-sql-kafka-0-10_2.11-2.4.4.jar:/home/cldr/streams-dev/libs/kafka-clients-2.0.0.jar") \
#         .config("spark.executor.extraLibrary", "/home/cldr/streams-dev/libs/spark-sql-kafka-0-10_2.11-2.4.4.jar:/home/cldr/streams-dev/libs/kafka-clients-2.0.0.jar") \
#         .config("spark.driver.extraClassPath", "/home/cldr/streams-dev/libs/spark-sql-kafka-0-10_2.11-2.4.4.jar:/home/cldr/streams-dev/libs/kafka-clients-2.0.0.jar") \
#         .getOrCreate()

spark.sparkContext.setLogLevel("DEBUG")

schema = StructType() \
        .add("WheelAngle", IntegerType()) \
        .add("acceleration", IntegerType()) \
        .add("heading", IntegerType()) \
        .add("reading_time", IntegerType()) \
        .add("tractionForce", IntegerType()) \
        .add("vel_latitudinal", IntegerType()) \
        .add("vel_longitudinal", IntegerType()) \
        .add("velocity", IntegerType()) \
        .add("x_pos", IntegerType()) \
        .add("y_pos", IntegerType()) \
        .add("yawrate", IntegerType())


 # Construct a streaming DataFrame that reads from testtopic
trans_det_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", brokers) \
        .option("subscribe", KAFKA_TOPIC) \
        .option("startingOffsets", "earliest") \
        .load() \
        .selectExpr("CAST(value as STRING)", "CAST(timestamp as STRING)","CAST(topic as STRING)")

print("Status " + str(trans_det_df.isStreaming))

#(from_json(col("value").cast("string"),schema))

    #Q1 =  trans_det_df.select(from_json(col("value"), schema).alias("parsed_value"), "timestamp")
    #Q2 =  trans_det_d.select("parsed_value*", "timestamp")


# query = trans_det_df.writeStream \
#             .format("console") \
#             .option("truncate","false") \
#             .start() 

query = trans_det_df \
    .writeStream \
    .format("memory") \
    .queryName("testData200") \
    .outputMode("append") \
    .start()
# from IPython.display import display, clear_output
# import time 
# while True:
#     clear_output(wait=True)
#     display(query.status )
#     display(spark.sql("select * from testData2").show())
#     time.sleep(5)

Status True


In [25]:
from IPython.display import display, clear_output
import time 
while True:
    clear_output(wait=True)
    display(query.status )
    raw = spark.sql("select * from testData200")
    raw.show()
    time.sleep(5)

{'message': 'Terminated with exception: Writing job aborted.',
 'isDataAvailable': False,
 'isTriggerActive': False}

+-----+---------+-----+
|value|timestamp|topic|
+-----+---------+-----+
+-----+---------+-----+



KeyboardInterrupt: 